In [1]:
import pandas as pd
from copy import deepcopy
import os
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from tqdm import tqdm
from glob import glob

import pwlf

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
speed_file = '/scratch/ab9738/traffic/traffic-jams-uber/results/speeds_nairobi.csv'
speed_df = pd.read_csv(speed_file)

In [4]:
def try_float(x):
    try:
        float(x)
        return True
    except:
        return False

speed_df = speed_df.loc[speed_df.min_speed.apply(try_float)]
speed_df = speed_df.loc[speed_df.max_speed.apply(try_float)]
speed_df = speed_df.loc[speed_df.s1.apply(try_float)]
speed_df = speed_df.loc[speed_df.s2.apply(try_float)]

speed_df['s1'] = speed_df['s1'].astype(float)
speed_df['s2'] = speed_df['s2'].astype(float)
speed_df['min_speed'] = speed_df['min_speed'].astype(float)
speed_df['max_speed'] = speed_df['max_speed'].astype(float)

speed_df

,osm_wayid,s1,s2,max_speed,min_speed
0,677812730,13.923265,20.757885,45.092,3.880
1,259886381,33.285743,68.223624,95.375,1.587
2,583039625,10.621541,29.451291,60.469,3.643
3,187197450,17.944687,37.758301,66.580,12.060
4,674984258,19.562451,35.270415,67.559,1.225
...,...,...,...,...,...
3080,532650195,26.741411,38.448295,65.297,12.822
3081,95792925,10.632660,21.225265,46.629,0.748
3082,239795145,13.248042,22.192110,75.087,9.528
3083,563011788,21.410131,29.021804,55.127,0.738


In [5]:
speed_df = speed_df[speed_df['min_speed']<speed_df['s1']]
speed_df = speed_df[speed_df['s1']<speed_df['s2']]
speed_df = speed_df[speed_df['s2']<speed_df['max_speed']]
speed_df = speed_df.reset_index(drop=True)

In [6]:
speed_df

,osm_wayid,s1,s2,max_speed,min_speed
0,677812730,13.923265,20.757885,45.092,3.880
1,259886381,33.285743,68.223624,95.375,1.587
2,583039625,10.621541,29.451291,60.469,3.643
3,187197450,17.944687,37.758301,66.580,12.060
4,674984258,19.562451,35.270415,67.559,1.225
...,...,...,...,...,...
3058,532650195,26.741411,38.448295,65.297,12.822
3059,95792925,10.632660,21.225265,46.629,0.748
3060,239795145,13.248042,22.192110,75.087,9.528
3061,563011788,21.410131,29.021804,55.127,0.738


In [7]:
# Speed Conversion for NYC

# speed_df = speed_df.assign(s1=speed_df['s1']*1.6)
# speed_df = speed_df.assign(s2=speed_df['s2']*1.6)
# speed_df = speed_df.assign(max_speed=speed_df['max_speed'].astype(float)*1.6)
# speed_df = speed_df.assign(min_speed=speed_df['min_speed'].astype(float)*1.6)

# def canbefloat(x):
#     try:
#         float(x)
#     except:
#         return False
#     else:
#         return True

# ucanbefloat = lambda t: canbefloat(t)
# vfunc = np.vectorize(ucanbefloat)

# speed_df = speed_df[vfunc(speed_df['min_speed'])]

# speed_df.to_csv('/scratch/ab9738/traffic/traffic-jams-uber/results/speeds_nyc.csv',index=False)

In [8]:
# Code to aggregate the jam_files for different segments.

# jam_folder = '/scratch/ab9738/traffic/traffic-jams-uber/results/jams_nairobi/'

# def read_jam_files(wayid):
#     jam_file = jam_folder+str(wayid)+'.csv'
#     jam_df_ = pd.read_csv(jam_file)
#     return(jam_df_)

# jam_dfs = Parallel(n_jobs=12)(delayed(read_jam_files)(wayid) for wayid in speed_df['osm_wayid'].values)

# jam_df = pd.concat(jam_dfs)

# jam_df = jam_df.drop('index',axis=1)

# jam_df.to_csv('/scratch/ab9738/traffic/traffic-jams-uber/results/jams_nairobi.csv')

In [9]:
jam_file = '/scratch/ab9738/traffic/traffic-jams-uber/results/jams_nairobi.csv'
jam_df = pd.read_csv(jam_file)

In [10]:
jam_df

,osm_way_id,utc_timestamp,speed_kph_mean,Jam_Duration
0,677812730,2018-01-31 18:00:00,13.039,0 days 00:00:00
1,677812730,2019-04-05 16:00:00,13.064,0 days 00:00:00
2,677812730,2019-05-08 17:00:00,13.131,0 days 00:00:00
3,677812730,2019-05-16 16:00:00,11.932,0 days 00:00:00
4,677812730,2019-06-28 18:00:00,11.762,0 days 00:00:00
...,...,...,...,...
4103225,563011788,2020-01-08 14:00:00,11.884,0 days 02:00:00
4103226,563011788,2020-01-09 14:00:00,10.890,0 days 01:00:00
4103227,563011788,2020-01-13 14:00:00,9.175,0 days 02:00:00
4103228,563011788,2020-01-24 11:00:00,10.797,0 days 01:00:00


In [11]:
jam_df = jam_df.reset_index(drop=True)

In [12]:
# import swifter
# def check_jam_th(row):
#     wayid = row.osm_way_id
#     s1 = speed_df[speed_df['osm_wayid']==wayid]['s1']
#     s2 = speed_df[speed_df['osm_wayid']==wayid]['s2']
#     jam_th = (s1.to_numpy()[0] + s2.to_numpy()[0])/4
#     if(row.speed_kph_mean<jam_th):
#         return True
#     else:
#         return False
# jam_df = jam_df[jam_df.swifter.apply(lambda row: check_jam_th(row), axis=1)]

In [13]:
# Adding max and min to speeds

# data_folder = '/scratch/ab9738/traffic/data/NYC/per-segment/'

# def canbefloat(x):
#     try:
#         float(x)
#     except:
#         return False
#     else:
#         return True

# def return_speed_limits(wayid):
#     ucanbefloat = lambda t: canbefloat(t)
#     vfunc = np.vectorize(ucanbefloat)
#     data_file = data_folder+str(wayid)+'.csv'
#     data_df_ = pd.read_csv(data_file,on_bad_lines='skip',header=None,low_memory=False)
#     cols_ = pd.read_csv('/scratch/ab9738/traffic/data/NYC/movement-speeds-hourly-new-york-2018-1.csv',nrows=1).columns
#     data_df_.columns = cols_
#     data_df_ = data_df_.dropna()
#     speed_col_ = data_df_['speed_mph_mean'].to_numpy()
#     speed_col_[vfunc(speed_col_)].astype(np.float)
#     max_speed_ = np.amax(speed_col_)
#     min_speed_ = np.amin(speed_col_)
#     return([max_speed_, min_speed_])

# speeds = Parallel(n_jobs=12)(delayed(return_speed_limits)(wayid) for wayid in speed_df['osm_wayid'].values)
# speeds = [x for x in speeds if x is not None]
# speeds = np.array(speeds)

# speed_df['max_speed'] = speeds[:,0]
# speed_df['min_speed'] = speeds[:,1]

# speed_df.to_csv('/scratch/ab9738/traffic/traffic-jams-uber/results/speeds_nyc.csv',index=False)

In [14]:
def canbefloat(x):
    try:
        float(x)
    except:
        return False
    else:
        return True

ucanbefloat = lambda t: canbefloat(t)
vfunc = np.vectorize(ucanbefloat)
tmp = jam_df['osm_way_id'].to_numpy()
jam_df = jam_df[vfunc(tmp)]
jam_df = jam_df.assign(osm_way_id=jam_df['osm_way_id'].astype(float))
jam_df.replace([np.inf, -np.inf], np.nan, inplace=True)
jam_df = jam_df.dropna()
jam_df = jam_df.assign(osm_way_id=jam_df['osm_way_id'].astype(int))

In [15]:
jam_df

,osm_way_id,utc_timestamp,speed_kph_mean,Jam_Duration
0,677812730,2018-01-31 18:00:00,13.039,0 days 00:00:00
1,677812730,2019-04-05 16:00:00,13.064,0 days 00:00:00
2,677812730,2019-05-08 17:00:00,13.131,0 days 00:00:00
3,677812730,2019-05-16 16:00:00,11.932,0 days 00:00:00
4,677812730,2019-06-28 18:00:00,11.762,0 days 00:00:00
...,...,...,...,...
4103225,563011788,2020-01-08 14:00:00,11.884,0 days 02:00:00
4103226,563011788,2020-01-09 14:00:00,10.890,0 days 01:00:00
4103227,563011788,2020-01-13 14:00:00,9.175,0 days 02:00:00
4103228,563011788,2020-01-24 11:00:00,10.797,0 days 01:00:00


In [16]:
speed_df

,osm_wayid,s1,s2,max_speed,min_speed
0,677812730,13.923265,20.757885,45.092,3.880
1,259886381,33.285743,68.223624,95.375,1.587
2,583039625,10.621541,29.451291,60.469,3.643
3,187197450,17.944687,37.758301,66.580,12.060
4,674984258,19.562451,35.270415,67.559,1.225
...,...,...,...,...,...
3058,532650195,26.741411,38.448295,65.297,12.822
3059,95792925,10.632660,21.225265,46.629,0.748
3060,239795145,13.248042,22.192110,75.087,9.528
3061,563011788,21.410131,29.021804,55.127,0.738


In [17]:
def canbefloat(x):
    try:
        float(x)
    except:
        return False
    else:
        return True

data_folder = '/scratch/ab9738/traffic/data/Nairobi/per-segment/'

def find_jam_duration(wayid_jam_df):
    
    try:
        wayid = wayid_jam_df.iloc[0].osm_way_id

        ucanbefloat = lambda t: canbefloat(t)
        vfunc = np.vectorize(ucanbefloat)

        s1 = speed_df[speed_df['osm_wayid']==wayid]['s1']
        s2 = speed_df[speed_df['osm_wayid']==wayid]['s2']
        jam_th = (s1.to_numpy()[0] + s2.to_numpy()[0])/4

        data_file = data_folder+str(wayid)+'.csv'
        df = pd.read_csv(data_file,on_bad_lines='skip',header=None,low_memory=False)
        cols = pd.read_csv('/scratch/ab9738/traffic/data/Nairobi/movement-speeds-hourly-nairobi-2018-1.csv',nrows=1).columns
        df.columns = cols
        df = df.dropna()

        timestamp = df['utc_timestamp']
        timestamp = pd.to_datetime(timestamp, format="%Y-%m-%dT%H:%M:%S.000Z", errors='coerce')
        df = df.assign(utc_timestamp=timestamp)
        df = df.dropna()
        df = df.sort_values(by='utc_timestamp')

        tmp = df['speed_kph_mean'].to_numpy()
        df = df[vfunc(tmp)]

        duration = []
        for j in range(len(wayid_jam_df)):
            start_time = pd.to_datetime(wayid_jam_df.iloc[j].utc_timestamp)
            # ts = wayid_jam_df.iloc[j].utc_timestamp
            ts = start_time
            speed_curr = float(wayid_jam_df.iloc[j].speed_kph_mean)
            i = np.where(df['utc_timestamp']==ts)[0][0]
            i_start = i
            while(speed_curr<jam_th and i<(len(df)-1)):
                i = i+1
                speed_curr = float(df.iloc[i].speed_kph_mean)
                ts = df.iloc[i].utc_timestamp
                if(ts-pd.Timedelta(hours=1)!=df.iloc[i-1].utc_timestamp):
                    break
            end_time = df.iloc[i-1].utc_timestamp
            i_end = i-1
            duration.append(i_end-i_start+1)

        wayid_jam_df_ = deepcopy(wayid_jam_df)
        wayid_jam_df_['Jam_Duration'] = np.array(duration)
    except:
        wayid_jam_df_ = deepcopy(wayid_jam_df)
        wayid_jam_df_['Jam_Duration'] = np.nan
    
    return(wayid_jam_df_)

In [18]:
sampled_jam_df = jam_df.sample(frac=0.1, random_state=17)

In [19]:
jam_dfs = Parallel(n_jobs=12)(delayed(find_jam_duration)(jam_df[jam_df['osm_way_id']==wayid])\
                              for wayid in speed_df['osm_wayid'].values if len(jam_df[jam_df['osm_way_id']==wayid])>1)

In [20]:
# jam_dfs = []
# for wayid in speed_df['osm_wayid'].values:
#     wayid_jam_df = jam_df[jam_df['osm_way_id']==wayid]
#     if(len(wayid_jam_df)):
#         res = find_jam_duration(wayid_jam_df)
#         jam_dfs.append(res)

In [21]:
jam_df_final = pd.concat(jam_dfs)

In [22]:
jam_df_final = jam_df_final.dropna()

In [23]:
# jam_df_final.to_csv('/scratch/ab9738/traffic/traffic-jams-uber/results/jams_sao_paulo.csv', index=False)

In [24]:
jam_df_final

,osm_way_id,utc_timestamp,speed_kph_mean,Jam_Duration
0,677812730,2018-01-31 18:00:00,13.039,0
1,677812730,2019-04-05 16:00:00,13.064,0
2,677812730,2019-05-08 17:00:00,13.131,0
3,677812730,2019-05-16 16:00:00,11.932,0
4,677812730,2019-06-28 18:00:00,11.762,0
...,...,...,...,...
4103225,563011788,2020-01-08 14:00:00,11.884,2
4103226,563011788,2020-01-09 14:00:00,10.890,1
4103227,563011788,2020-01-13 14:00:00,9.175,2
4103228,563011788,2020-01-24 11:00:00,10.797,1


In [25]:
jam_df_final.set_index('osm_way_id').loc[555497908]

,utc_timestamp,speed_kph_mean,Jam_Duration
osm_way_id,,,
555497908,2018-01-15 13:00:00,12.179,2
555497908,2018-01-25 14:00:00,13.176,1
555497908,2018-02-01 13:00:00,13.424,1
555497908,2018-02-20 13:00:00,13.394,1
555497908,2018-05-02 12:00:00,12.176,2
555497908,2018-06-14 14:00:00,12.981,1
555497908,2019-03-27 13:00:00,7.511,1
555497908,2019-05-13 13:00:00,8.153,1
555497908,2019-06-27 13:00:00,11.110,1


In [25]:
# jam_df_final_ = jam_df_final[jam_df_final['Jam_Duration'].dt.components['days']==0]

In [28]:
np.where(jam_df_final['Jam_Duration'].to_numpy()==67)

(array([3200580]),)

In [41]:
jam_df_final.iloc[3200580].utc_timestamp

'2019-12-06 05:00:00'

In [35]:
speed_df.set_index('osm_wayid').loc[43270176]

s1            47.418837
s2            71.970238
max_speed    104.358000
min_speed      4.345000
Name: 43270176, dtype: float64

In [36]:
ucanbefloat = lambda t: canbefloat(t)
vfunc = np.vectorize(ucanbefloat)

data_file = '/scratch/ab9738/traffic/data/Nairobi/per-segment/43270176.csv'
df = pd.read_csv(data_file,on_bad_lines='skip',header=None,low_memory=False)
cols = pd.read_csv('/scratch/ab9738/traffic/data/Nairobi/movement-speeds-hourly-nairobi-2018-1.csv',nrows=1).columns
df.columns = cols
df = df.dropna()

timestamp = df['utc_timestamp']
timestamp = pd.to_datetime(timestamp, format="%Y-%m-%dT%H:%M:%S.000Z", errors='coerce')
df = df.assign(utc_timestamp=timestamp)
df = df.dropna()
df = df.sort_values(by='utc_timestamp')

tmp = df['speed_kph_mean'].to_numpy()
df = df[vfunc(tmp)]

In [43]:
df = df.reset_index(drop=True)

In [45]:
df[df['utc_timestamp']==pd.to_datetime(jam_df_final.iloc[3200580].utc_timestamp)]

,year,month,day,hour,utc_timestamp,segment_id,start_junction_id,end_junction_id,osm_way_id,osm_start_node_id,osm_end_node_id,speed_kph_mean,speed_kph_stddev
10081,2019,12,6,8,2019-12-06 05:00:00,8321067dbc806d5df6a77fadcffd516534ec5fba,692da53053446f27363eab14bc9ee07c9db3c990,3c6d72bfe62a7f76e6cf5bddff531d7de03b64c7,43270176.0,1.473397e+09,543399884.0,14.391,7.421


In [48]:
df.iloc[10080:10120][['utc_timestamp', 'speed_kph_mean']]

,utc_timestamp,speed_kph_mean
10080,2019-12-06 03:00:00,16.750
10081,2019-12-06 05:00:00,14.391
10082,2019-12-06 06:00:00,12.727
10083,2019-12-06 07:00:00,10.421
10084,2019-12-06 08:00:00,15.672
10085,2019-12-06 09:00:00,28.670
10086,2019-12-06 10:00:00,22.167
10087,2019-12-06 11:00:00,16.572
10088,2019-12-06 12:00:00,10.486
10089,2019-12-06 13:00:00,10.998
